<a href="https://colab.research.google.com/github/suhbaca/AI4GOOD_TeamProject/blob/suh/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **LLAMA 2 Model**





In [1]:
#access huggingface
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [3]:
#implement a pipeline that will allow us to give prompts and get responses

import torch
import transformers

pipeline = transformers.pipeline(
"text-generation",

model=model,

tokenizer=tokenizer,

torch_dtype=torch.float16,

device_map="auto",

)

In [46]:
# create a sequence that will take in a prompt

#changing the num_return_sequence will give 3 different possible outputs for the same prompt

sequences = pipeline(
'I am looking for a word for something both round and sweet to taste. Give me 10 possible words and no other text included. Give me the words from the most probable word to the least probable',

do_sample=True,

top_k=100,

num_return_sequences=3,

eos_token_id=tokenizer.eos_token_id,

max_length=500,

temperature= 0.3

)
print(sequences)
for seq in sequences:
  print(1)
  print(f"{seq['generated_text']}")

[{'generated_text': 'I am looking for a word for something both round and sweet to taste. Give me 10 possible words and no other text included. Give me the words from the most probable word to the least probable.\n\n1. Doughnut\n2. Muffin\n3. Cake\n4. Pastry\n5. Scone\n6. Tart\n7. Pudding\n8. Custard\n9. Pies\n10. Tarts\n\nPlease let me know if you need any more information.'}, {'generated_text': 'I am looking for a word for something both round and sweet to taste. Give me 10 possible words and no other text included. Give me the words from the most probable word to the least probable word.\n\n1. Candy\n2. Muffin\n3. Donut\n4. Pastry\n5. Tart\n6. Pudding\n7. Cake\n8. Cookie\n9. Biscuit\n10. Scone\n\nPlease let me know if you need any further information.'}, {'generated_text': 'I am looking for a word for something both round and sweet to taste. Give me 10 possible words and no other text included. Give me the words from the most probable word to the least probable word.\n\n1. Candy\n2.